In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_Box_sns
import GrubbsTest

In [ ]:
# NOTE: NULL is also possible
possible_enddeviceeventtypeids = [
    '3.1.3.79',
    '3.2.0.28',
    '3.2.0.85',
    '3.2.22.28',
    '3.2.22.150',
    '3.7.19.242',
    '3.7.19.243',
    '3.7.22.4',
    '3.7.22.19',
    '3.8.0.215',
    '3.9.83.159',
    '3.11.63.161',
    '3.12.0.257',
    '3.12.17.257',
    '3.12.48.28',
    '3.12.48.219',
    '3.12.93.28',
    '3.12.93.219',
    '3.12.136.38',
    '3.12.136.85',
    '3.18.1.199',
    '3.18.1.220',
    '3.18.72.28',
    '3.18.72.79',
    '3.18.72.85',
    '3.18.85.28',
    '3.18.85.79',
    '3.18.85.85',
    '3.18.92.28',
    '3.18.92.79',
    '3.18.92.85',
    '3.21.1.79',
    '3.21.1.173',
    '3.21.3.28',
    '3.21.3.79',
    '3.21.17.28',
    '3.21.18.79',
    '3.21.67.28',
    '3.21.67.79',
    '3.21.82.28',
    '3.21.82.79',
    '3.22.12.243',
    '3.22.19.242',
    '3.23.17.79',
    '3.23.17.139',
    '3.23.136.47',
    '3.23.136.85',
    '3.25.17.3',
    '3.26.0.47',
    '3.26.0.216',
    '3.26.17.185',
    '3.26.17.216',
    '3.26.38.27',
    '3.26.38.37',
    '3.26.38.47',
    '3.26.38.73',
    '3.26.38.93',
    '3.26.38.150',
    '3.26.136.47',
    '3.26.136.66',
    '3.26.136.216',
    '3.31.1.143',
    '3.33.1.219',
    '3.33.1.257',
    '3.35.0.28',
    '3.35.0.40',
    '3.36.0.79',
    '3.36.1.29',
    '3.36.114.73',
    '3.36.114.159',
    '3.36.136.73',
    '3.36.136.79',
    '3.38.1.139'      
]

In [ ]:
conn_outages = Utilities.get_utldb01p_oracle_connection()
conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
fig_num = 0

# ---------------------------------------------------------------
# OUTAGES
# ---------------------------------------------------------------

In [ ]:
sql_outage = DOVSOutages_SQL.build_sql_std_outage(
    specific_dates=['2020-10-12','2017-09-16','2020-07-01'],                                    
    outg_rec_nbs=['11947217','11946640','10143524','11770899']
)
sql_outage.print()

# NOTE
11946344 is scheduled output.  So, there are apparently no corresponding end events!

In [ ]:
dovs_outgs = DOVSOutages(df_construct_type=DFConstructType.kRunSqlQuery, 
                         build_sql_function_kwargs=dict(
                             specific_dates=['2020-10-12','2017-09-16','2020-07-01'], 
                             outg_rec_nbs=['11947217','11946640','10143524','11770899']
                         ), 
                         init_df_in_constructor=True)
df_outage = dovs_outgs.get_df().copy()

In [ ]:
#TODO
#search_time_window = 60 #minutes
search_time_window = 60*24*14 #minutes
search_time_window = datetime.timedelta(minutes=search_time_window)

# NOTE: search_time_window can also be a vector of values...
df_outage['search_time_window'] = search_time_window
df_outage['t_search_min'] = df_outage['DT_OFF_TS_FULL'] - df_outage['search_time_window']
df_outage['t_search_max'] = df_outage['DT_ON_TS'] + df_outage['search_time_window']

# ---------------------------------------------------------------
# END DEVICE EVENTS
# ---------------------------------------------------------------

How to generalize?
I'll need to do something like joining on premise numbers together with the approximate time of the outage.
<br> Will probably use +-1 hr from outage to begin?

In [ ]:
cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
cols_of_interest_met_prem = TableInfos.MeterPremise_TI.std_columns_of_interest

In [ ]:
# df_outage.groupby(['DT_ON_TS', 'DT_OFF_TS_FULL']).ngroups

In [ ]:
sql_end_events_join_mp_for_outages = AMIEndEvents_SQL.build_sql_end_events_for_outages(
    cols_of_interest=cols_of_interest_end_dev_event, 
    df_outage=df_outage, 
    build_sql_function_kwargs=dict(opco='oh', 
                        aep_derived_uoms_and_idntfrs=['KWH']), 
    join_mp_args=dict(join_with_CTE=True, 
                      build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem))
)
print(sql_end_events_join_mp_for_outages)

In [ ]:
# sql_end_events_join_mp_for_outages = AMIEndEvents_SQL.build_sql_end_events_for_outages(
#     cols_of_interest = cols_of_interest_end_dev_event, 
#     df_outage=df_outage, 
#     build_sql_function_kwargs=dict(opco='oh', 
#                                    aep_derived_uoms_and_idntfrs=['KWH']), 
#     join_mp_args=dict(join_with_CTE=True, 
#                       build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem))    
# )
# print(sql_end_events_join_mp_for_outages)

In [ ]:
# test_df = pd.read_sql(sql_end_events_join_mp_for_outages, conn_aws)
# test_df = Utilities_df.remove_table_aliases(test_df)

In [ ]:
# test_df

In [ ]:
# sql_stmnt_end_dev_event = AMIEndEvents_SQL.build_sql_end_events_for_outages(cols_of_interest_end_dev_event, df_outage)
# print(sql_stmnt_end_dev_event)

# BEGIN DELETE ME

In [ ]:
# def build_sql_end_events(cols_of_interest_end_dev_event, 
#                          **kwargs):
#     r"""
#     Acceptable kwargs:
#       - date_range
#         - tuple with two string elements, e.g., ['2021-01-01', '2021-04-01']
#       - aep_event_dt_col    
      
#       - serial_numbers
#       - serialnumber_col

#       - aep_premise_nbs
#       - aep_premise_nb_col

#       - aep_opco
#       - aep_opco_col
      
#       - schema_name
#       - table_name

In [ ]:
t_search_min = '2020-09-28 19:18:00' 
t_search_max = '2020-10-26 20:45:00'
aep_premise_nbs = ['100420620','100547600','100683783','101020620','101330620','103520620','103647600','103783783',
                   '104120620','104430620','105030620','105293783','105710620','106620620','106747600','106883783',
                   '107220620','107483783','108130620','108393783','108810620','109720620','109847600']
date_range = [pd.to_datetime(t_search_min).date(), pd.to_datetime(t_search_max).date()]

In [ ]:
import AMI_SQL
reload(AMI_SQL)
from AMI_SQL import AMI_SQL

import AMIEndEvents_SQL
reload(AMIEndEvents_SQL)
from AMIEndEvents_SQL import AMIEndEvents_SQL

In [ ]:
ede_i = AMIEndEvents_SQL.build_sql_end_events(cols_of_interest_end_dev_event, date_range=date_range, aep_premise_nbs=aep_premise_nbs)
ede_i.alias = 'ede_i'

In [ ]:
ede_i.print(include_alias=True)

In [ ]:
ami_end_events = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_outages, 
    build_sql_function_kwargs = dict(
        cols_of_interest=cols_of_interest_end_dev_event, 
        df_outage=df_outage
    ), 
    contstruct_df_args=None, 
    init_df_in_constructor=True
)
#df_2 = ami_end_events.df()
df_2 = ami_end_events.df
df_2 = Utilities_df.remove_table_aliases(df_2)

In [ ]:
sql_stmnt_end_dev_event = AMIEndEvents_SQL.build_sql_end_events_for_outages(cols_of_interest_end_dev_event, df_outage)
start = time.time()
df_1 = pd.read_sql(sql_stmnt_end_dev_event, conn_aws)
print(time.time()-start)
df_1 = Utilities_df.remove_table_aliases(df_1)

In [ ]:
df_2.equals(df_1)

In [ ]:
sql_end_events_join_mp_for_outages = AMIEndEvents_SQL.build_sql_end_events_for_outages(
    cols_of_interest=cols_of_interest_end_dev_event, 
    df_outage=df_outage, 
    build_sql_function_kwargs=dict(opco='oh', 
                        aep_derived_uoms_and_idntfrs=['KWH']), 
    join_mp_args=dict(join_with_CTE=True, 
                      build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem))
)
df_1 = pd.read_sql(sql_end_events_join_mp_for_outages, conn_aws)
df_1 = Utilities_df.remove_table_aliases(df_1)

In [ ]:
ami_end_events = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_outages, 
    build_sql_function_kwargs = dict(
        cols_of_interest=cols_of_interest_end_dev_event, 
        df_outage=df_outage, 
        build_sql_function_kwargs = dict(
            opco='oh',
            aep_derived_uoms_and_idntfrs=['KWH']
        ), 
        join_mp_args=dict(join_with_CTE=True, 
                          build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem))
    ), 
    contstruct_df_args=None, 
    init_df_in_constructor=True
)
#df_2 = ami_end_events.df()
df_2 = ami_end_events.df
df_2 = Utilities_df.remove_table_aliases(df_2)

In [ ]:
df_2.equals(df_1)

In [ ]:
sql_end_dev_event_old = AMIEndEvents_SQL.build_sql_end_events_for_outages(cols_of_interest_end_dev_event, df_outage, split_to_CTEs=False)
sql_stmnt_end_dev_event = AMIEndEvents_SQL.build_sql_end_events_for_outages(cols_of_interest_end_dev_event, df_outage)

In [ ]:
start = time.time()
new_df = pd.read_sql(sql_stmnt_end_dev_event, conn_aws)
print(time.time()-start)
new_df = Utilities_df.remove_table_aliases(new_df)

In [ ]:
start = time.time()
old_df = pd.read_sql(sql_end_dev_event_old, conn_aws)
print(time.time()-start)
old_df = Utilities_df.remove_table_aliases(old_df)

In [ ]:
time.sleep(10)

In [ ]:
start = time.time()
old_df = pd.read_sql(sql_end_dev_event_old, conn_aws)
print(time.time()-start)
old_df = Utilities_df.remove_table_aliases(old_df)

In [ ]:
start = time.time()
new_df = pd.read_sql(sql_stmnt_end_dev_event, conn_aws)
print(time.time()-start)
new_df = Utilities_df.remove_table_aliases(new_df)

In [ ]:
sort_by = ['issuertracking_id', 'aep_event_dt', 'valuesinterval', 'serialnumber']

In [ ]:
old_df.sort_values(by=sort_by, ignore_index=True).equals(new_df.drop(columns=['outg_rec_nb']).sort_values(by=sort_by, ignore_index=True))

In [ ]:
print(sql_end_dev_event_old)

In [ ]:
print(sql_stmnt_end_dev_event)

### See if matching procedure agrees with new SQL procedure which already includes the outage number

In [ ]:
old_df = Utilities_dt.build_utc_time_column(old_df, time_col='valuesinterval')
old_df = Utilities_dt.strip_tz_info_and_convert_to_dt(old_df, time_col='valuesinterval', 
                                                      placement_col='valuesinterval_local', run_quick=True)

start = time.time()
old_df, n_events_with_mult_matches = clm.match_events_in_df_to_outages(old_df, df_outage, 
                                                                       return_n_events_with_mult_matches=True)
old_df = clm.set_all_outages_info_in_events_df(old_df, df_outage)
time_new = time.time() - start
print(f'n_events_with_mult_matches = {n_events_with_mult_matches}')

In [ ]:
new_df = Utilities_dt.build_utc_time_column(new_df, time_col='valuesinterval')
new_df = Utilities_dt.strip_tz_info_and_convert_to_dt(new_df, time_col='valuesinterval', 
                                                      placement_col='valuesinterval_local', run_quick=True)
new_df['outg_rec_nb'] = new_df['outg_rec_nb'].astype(int)

In [ ]:
old_df = old_df.drop(columns=['OUTG_REC_NB_OLD', 'OUTG_REC_NB_ALL', 'n_outages_found',
                              'DT_OFF_TS_FULL', 'DT_ON_TS', 'STEP_DRTN_NB',
                              'search_time_window'])
old_df = old_df.rename(columns={'OUTG_REC_NB':'outg_rec_nb'})
old_df = old_df[new_df.columns]
old_df['outg_rec_nb'] = old_df['outg_rec_nb'].astype(int)

In [ ]:
old_df.head()

In [ ]:
new_df.head()

In [ ]:
old_df.sort_values(by=sort_by, ignore_index=True).equals(new_df.sort_values(by=sort_by, ignore_index=True))

# END DELETE ME

In [ ]:
# start = time.time()
# df_end_dev_event_OLD = pd.read_sql(sql_end_dev_event_OLD, conn_aws)
# time_OLD = time.time() - start
# print(f'time_OLD = {time_OLD}')
# print(f"len(df_end_dev_event_OLD['serialnumber'].unique()) = {len(df_end_dev_event_OLD['serialnumber'].unique())}")

start = time.time()
df_end_dev_event = pd.read_sql(sql_stmnt_end_dev_event, conn_aws)
time_NEW = time.time() - start
df_end_dev_event = Utilities_df.remove_table_aliases(df_end_dev_event)
print(f'time_NEW = {time_NEW}')
print(f"len(df_end_dev_event['serialnumber'].unique())     = {len(df_end_dev_event['serialnumber'].unique())}")

df_end_dev_event = Utilities_df.remove_prepend_from_columns_in_df(df_end_dev_event)

In [ ]:
# TODO no real point to building valuesinterval_utc, as the outages are in local time
# Maybe want to change outages to UTC time instead?  No immediately clear simple method
# to achieve this through fields in DOVS tables.
df_end_dev_event = Utilities_dt.build_utc_time_column(df_end_dev_event, time_col='valuesinterval')
df_end_dev_event = Utilities_dt.strip_tz_info_and_convert_to_dt(df_end_dev_event, time_col='valuesinterval', 
                                                   placement_col='valuesinterval_local', run_quick=True)

In [ ]:
df_end_dev_event_cpy = df_end_dev_event.copy()
start = time.time()
df_end_dev_event_cpy, n_events_with_mult_matches = clm.match_events_in_df_to_outages(df_end_dev_event_cpy, df_outage, 
                                                                                      return_n_events_with_mult_matches=True)
df_end_dev_event_cpy = clm.set_all_outages_info_in_events_df(df_end_dev_event_cpy, df_outage)
time_new = time.time() - start
print(f'n_events_with_mult_matches = {n_events_with_mult_matches}')

In [ ]:
# Need to be a little careful with n_events_with_mult_matches below, as values were exploded in match_events_in_df_to_outages
# Therefore, naively taking df_end_dev_event_cpy[df_end_dev_event_cpy['n_outages_found'] > 1].shape[0] would overcount.
#   e.g., if every event with 'n_outages_found'>1 found two outages, the value returned by the naive method mentioned
#         above would be twice the actual value
n_events_with_mult_matches = len(df_end_dev_event_cpy[df_end_dev_event_cpy['n_outages_found'] > 1]['issuertracking_id'].unique())
n_events_with_no_matches = df_end_dev_event_cpy[df_end_dev_event_cpy['n_outages_found'] < 1].shape[0]
print(f'n_events_with_mult_matches = {n_events_with_mult_matches}')
print(f'n_events_with_no_matches = {n_events_with_no_matches}')

In [ ]:
outg_rec_nb = df_end_dev_event_cpy['OUTG_REC_NB'].unique().tolist()[0]
tmp_df = df_end_dev_event_cpy[df_end_dev_event_cpy['OUTG_REC_NB']==outg_rec_nb]

assert(len(tmp_df['DT_OFF_TS_FULL'].unique())==1)
assert(len(tmp_df['DT_ON_TS'].unique())==1)
out_beg = tmp_df.iloc[0]['DT_OFF_TS_FULL']
out_end = tmp_df.iloc[0]['DT_ON_TS']
expand_time = datetime.timedelta(days=2)

tmp_df = tmp_df.set_index('valuesinterval_local', drop=False).sort_index()
tmp_df.index.name='time_idx'
tmp_df = tmp_df[out_beg-expand_time:out_end+expand_time]

bin_size = '5min'
bins = pd.date_range(start = tmp_df['valuesinterval_local'].min().floor(bin_size), 
                     end = tmp_df['valuesinterval_local'].max().ceil(bin_size), 
                     freq=bin_size)

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, num=fig_num, figsize=[11, 8.5])
tmp_df.hist(ax=ax0, column='valuesinterval_local', bins=bins)
ax0.axvline(x=out_beg, color='red')
ax0.axvline(x=out_end, color='lawngreen')
ax0.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
ax0.set_title('Full')


tmp_df[tmp_df['valuesinterval_local'] < out_beg].hist(ax=ax1, column='valuesinterval_local', bins=bins)
ax1.axvline(x=out_beg, color='red')
ax1.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
ax1.set_title('Before Outage')


tmp_df[tmp_df['valuesinterval_local'] >= out_end].hist(ax=ax2, column='valuesinterval_local', bins=bins)
ax2.axvline(x=out_end, color='lawngreen')
ax2.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
ax2.set_title('After Outage')


plt.suptitle(f'outg_rec_nb: {outg_rec_nb}', fontsize=25, fontweight='bold')
fig_num += 1

In [ ]:
outg_rec_nb = df_end_dev_event_cpy['OUTG_REC_NB'].unique().tolist()[0]
tmp_df = df_end_dev_event_cpy[df_end_dev_event_cpy['OUTG_REC_NB']==outg_rec_nb]

assert(len(tmp_df['DT_OFF_TS_FULL'].unique())==1)
assert(len(tmp_df['DT_ON_TS'].unique())==1)
out_beg = tmp_df.iloc[0]['DT_OFF_TS_FULL']
out_end = tmp_df.iloc[0]['DT_ON_TS']

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, num=fig_num, figsize=[11, 8.5])
tmp_df.hist(ax=ax0, column='valuesinterval_local')
ax0.axvline(x=out_beg, color='red')
ax0.axvline(x=out_end, color='lawngreen')
ax0.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
ax0.set_title('Full')


tmp_df[tmp_df['valuesinterval_local'] < '2020-10-12 15:00:00'].hist(ax=ax1, column='valuesinterval_local')
ax1.axvline(x=out_beg, color='red')
ax1.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
ax1.set_title('Before Outage')


tmp_df[tmp_df['valuesinterval_local'] >= '2020-10-12 15:00:00'].hist(ax=ax2, column='valuesinterval_local')
ax2.axvline(x=out_end, color='lawngreen')
ax2.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
ax2.set_title('After Outage')


plt.suptitle(f'outg_rec_nb: {outg_rec_nb}', fontsize=25, fontweight='bold')
fig_num += 1

In [ ]:
outg_rec_nbs = df_end_dev_event_cpy['OUTG_REC_NB'].unique().tolist()
for outg_rec_nb in outg_rec_nbs:
    tmp_df = df_end_dev_event_cpy[df_end_dev_event_cpy['OUTG_REC_NB']==outg_rec_nb]
    
    assert(len(tmp_df['DT_OFF_TS_FULL'].unique())==1)
    assert(len(tmp_df['DT_ON_TS'].unique())==1)
    out_beg = tmp_df.iloc[0]['DT_OFF_TS_FULL']
    out_end = tmp_df.iloc[0]['DT_ON_TS']
    
    bin_size = '5min'
    bins = pd.date_range(start = tmp_df['valuesinterval_local'].min().floor(bin_size), 
                         end = tmp_df['valuesinterval_local'].max().ceil(bin_size), 
                         freq=bin_size)
    
    fig, (ax0, ax1, ax2) = plt.subplots(3, 1, num=fig_num, figsize=[11, 8.5])
    tmp_df.hist(ax=ax0, column='valuesinterval_local', bins=bins)
    ax0.axvline(x=out_beg, color='red')
    ax0.axvline(x=out_end, color='lawngreen')
    ax0.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
    ax0.set_title('Full')


    tmp_df[(tmp_df['valuesinterval_local'] < tmp_df['DT_OFF_TS_FULL'] + 0.5*pd.to_timedelta(tmp_df['DT_ON_TS']-tmp_df['DT_OFF_TS_FULL'])) & 
           (tmp_df['valuesinterval_local'] >= tmp_df['DT_OFF_TS_FULL'] - tmp_df['search_time_window'])].hist(ax=ax1, column='valuesinterval_local', bins=bins)
    ax1.axvline(x=out_beg, color='red')
    ax1.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
    ax1.set_title('Before Outage')


    tmp_df[(tmp_df['valuesinterval_local'] < tmp_df['DT_ON_TS'] + tmp_df['search_time_window']) & 
           (tmp_df['valuesinterval_local'] >= tmp_df['DT_ON_TS'] - 0.5*pd.to_timedelta(tmp_df['DT_ON_TS']-tmp_df['DT_OFF_TS_FULL']))].hist(ax=ax2, column='valuesinterval_local', bins=bins)
    ax2.axvline(x=out_end, color='lawngreen')
    ax2.tick_params(axis='x', labelrotation=0, labelsize=7.0, direction='in')
    ax2.set_title('After Outage')


    plt.suptitle(f'outg_rec_nb: {outg_rec_nb}', fontsize=25, fontweight='bold')
    fig_num += 1
